<a href="https://colab.research.google.com/github/ny0y/CustomerSupportChatbot/blob/main/ChatbotCustomerService.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi


Wed Dec  4 08:13:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P8              15W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Cell 1: Install Required Packages**

In [2]:
# Import required libraries
!pip install transformers
!pip install torch
!pip install pandas
!pip install scikit-learn



**Cell 2: Import Libraries**

In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import random
import re



**Cell 3: Load Dataset**




In [4]:
dataset_path = '/content/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv'
data = pd.read_csv(dataset_path)

data.head()  # Display first few rows for inspection

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


**Cell 4: Preprocess Data**

In [5]:
# Convert labels to integers (for BERT)
data['intent'] = data['intent'].astype('category').cat.codes

# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['instruction'], data['intent'], test_size=0.2, stratify=data['intent']
)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert to PyTorch tensors and make sure labels are of type Long (int64)

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels.values, dtype=torch.long)  # Make sure labels are of type Long
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels.values, dtype=torch.long)  # Make sure labels are of type Long
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


**Cell 5: Create DataLoader for Training**

In [6]:
# Create DataLoaders for training and testing
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

**Cell 6: Build the Model**

In [7]:
# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=27)

# Step 1: Check if GPU is available and move model to the device
device = torch.device('cuda')
model = model.to(device)

# Check the model device
print(f"Model is on device: {next(model.parameters()).device}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model is on device: cuda:0


**Cell 7: Train the Model**

In [8]:
# Initialize loss function
loss_fn = CrossEntropyLoss()

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5) # You can adjust the learning rate

# Training loop
for epoch in range(3):  # You can change the number of epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_dataloader, desc=f"\nEpoch {epoch + 1}"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Calculate loss
        loss = loss_fn(logits, labels)
        total_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Accuracy calculation
        _, preds = torch.max(logits, dim=1)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)

    avg_loss = total_loss / len(train_dataloader)
    accuracy = correct_predictions / total_predictions
    print(f"\tLoss = {avg_loss:.4f}, Accuracy = {accuracy:.4f}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

Epoch 1:   7%|▋         | 49/672 [00:11<02:15,  4.58it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x7981b5441f50>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1196, in __iter__
    self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1290, in close
    fp_write('')
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1287, in fp_write
    self.fp.write(str(s))
  File "/usr/local/lib/python3.10/dist-packages/tqdm/utils.py", line 196, in inner
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/iostream.py", line 402, in write
    se

KeyboardInterrupt: 

**Cell 8: Evaluate the Model**

In [ ]:
# Set model to evaluation mode
model.eval()

# Initialize variables to calculate accuracy
correct_predictions = 0
total_predictions = 0

# Disable gradient calculation for inference
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        # Move batch to device
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Accuracy calculation
        _, preds = torch.max(logits, dim=1)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_predictions
print(f"\tTest Accuracy: {accuracy:.4f}")


**Cell 9: Save and Load the Model**

In [ ]:
# Save the model and tokenizer
model.save_pretrained("/content/bert_chatbot_model")
tokenizer.save_pretrained("/content/bert_chatbot_model")

# Reload the model and tokenizer
model = BertForSequenceClassification.from_pretrained("/content/bert_chatbot_model")
tokenizer = BertTokenizer.from_pretrained("/content/bert_chatbot_model")

**Cell 10: Chatbot Inference**

In [ ]:
# Sample input text for inference
text_input = "How i do change my address"

# Tokenize the input text
inputs = tokenizer(text_input, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Move input tensors to the same device as the model (GPU or CPU)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Ensure the model is on the same device
model.to(device)

# Set model to evaluation mode
model.eval()

# Perform inference
with torch.no_grad():  # Disable gradients during inference
    outputs = model(**inputs)  # Forward pass through the model
    logits = outputs.logits  # Get the model's raw output (logits)
    predicted_class = torch.argmax(logits, dim=1).item()  # Get the predicted class index

# Debugging: Print the predicted class index and available categories
print(f"Predicted class index: {predicted_class}")

# Get the number of intent classes
num_classes = len(data['intent'].astype('category').cat.categories)
print(f"Number of classes: {num_classes}")

# Define the intent_labels list (make sure this list has the correct number of categories)
intent_labels = [
    "cancel_order", "change_order", "change_shipping_address", "check_cancellation_fee",
    "check_invoice", "check_payment_methods", "check_refund_policy", "complaint",
    "contact_customer_service", "contact_human_agent", "create_account", "delete_account",
    "delivery_options", "delivery_period", "edit_account", "get_invoice", "get_refund",
    "newsletter_subscription", "payment_issue", "place_order", "recover_password",
    "registration_problems", "review", "set_up_shipping_address", "switch_account",
    "track_order", "track_refund"
]

# Ensure the list has the correct number of categories
if len(intent_labels) == num_classes:
    predicted_intent = intent_labels[predicted_class]
    print(f"Chatbot Response: {predicted_intent}")
else:
    print(f"Error: Mismatch between predicted class index and intent labels list size.")


**Cell 11 : confusion matrix**

In [ ]:
# Get predictions
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predictions
        _, preds = torch.max(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Generate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=data['intent'].astype('category').cat.categories, yticklabels=data['intent'].astype('category').cat.categories)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

**Cell 12 : Test the classfication**

In [ ]:
# Sample code for user input
while True:
    # Prompt the user for input
    text_input = input("Please enter your message (type 'exit' to stop): ")

    # Exit condition
    if text_input.lower() == 'exit':
        print("Exiting the chatbot.")
        break

    # Tokenize the input text
    inputs = tokenizer(text_input, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Move input tensors to the same device as the model (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Ensure the model is on the same device
    model.to(device)

    # Set model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():  # Disable gradients during inference
        outputs = model(**inputs)  # Forward pass through the model
        logits = outputs.logits  # Get the model's raw output (logits)
        predicted_class = torch.argmax(logits, dim=1).item()  # Get the predicted class index

    # Map the predicted class index to the corresponding intent label
    predicted_intent = intent_labels[predicted_class]

    # Print the chatbot's response (predicted intent)
    print(f"Chatbot Response: {predicted_intent}")


**Cell 13: Real-Time Interaction **

In [9]:
df = pd.read_csv(dataset_path)

# Real-time chatbot interaction
while True:
    # Prompt the user for input
    text_input = input("\nfCustomer: Please enter your message (type 'exit' to stop): ")

    # Exit condition
    if text_input.lower() == 'exit':
        print("Exiting the chatbot. Goodbye!")
        break

    # Tokenize the input text
    inputs = tokenizer(text_input, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Move input tensors to the same device as the model (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Set model to evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():  # Disable gradients during inference
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    # Map the predicted class index to the corresponding intent label
    predicted_intent = intent_labels[predicted_class]

    # Filter the dataset to get responses for the predicted intent
    filtered_responses = df[df['intent'] == predicted_intent]['response'].tolist()

    # Generate a random response from the filtered responses
    if filtered_responses:
        response = random.choice(filtered_responses)
    else:
        response = "I'm sorry, I didn't understand your request. Can you please rephrase it?"

    # Output the chatbot's response
    print(f"Chatbot: {response}")


fCustomer: Please enter your message (type 'exit' to stop): fg


NameError: name 'intent_labels' is not defined